## Lib import

In [ ]:
import os
from typing import List, Literal, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver

## API key get

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# key
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# deepseek
deepseek_llm_model = "deepseek-chat"

# silicon
silicon_base_url =  "https://api.siliconflow.cn/v1"
silicon_llm_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = "sentence-transformers/all-MiniLM-L6-v2"

## LLM Init

In [ ]:
# init LLM mod
llm = ChatOpenAI(
    model=silicon_llm_model,
    openai_api_key=silicon_api_key,
    base_url=silicon_base_url,
    temperature=0.8,
    max_tokens=2048,
)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate

# init LLM mod
llm = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

## Prompt

In [ ]:
# Define a prompt template
template = """
You are an AI assistant. Your task is to help the user with their queries.

User: {user_input}
AI:
"""

# Create a PromptTemplate instance
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template=template,
)

# Example usage
user_input = "What is the capital of France?"
prompt = prompt_template.format(user_input=user_input)
print(prompt)

### chat prompt template

In [ ]:
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to Calerie Health and the healthcare products it produces.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

route_prompt.pretty_print()

In [ ]:
system = """You are a re-writer that converts a retrieved document to a better version that is optimized.
Rewrite the part of the document that relates to the user question and output them.
The output will not exceed 100 characters."""

re_write_document_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "User question: {question} \n Here is the original retrieved document: {document} \n\n Formulate an improved document."
        ),
    ]
)


re_write_document_prompt.pretty_print()   

### Deepseek Chat Prompt Template
Only use human and do not use system prompt.

In [ ]:
################################################################################
### Deepseek Question Re-writer
################################################################################
re_write_system_ds = """
Role: Prompt Optimization Specialist
Task: re-write input question to a better version

Guidelines:
You are a question re-writer that converts an input question to a better version that is optimized \
for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning.
"""

re_write_prompt_ds = ChatPromptTemplate.from_messages(
    [
        ("system", ""),
        (
            "human",
            f"[系统指令] \n {re_write_system_ds} \n\n [用户请求] \n Here is the initial question: \n {{question}} \n\n Formulate an improved question."
        ),
    ]
)

re_write_prompt_ds.pretty_print()  

In [ ]:
document_rewriter = re_write_prompt_ds | llm | StrOutputParser()
doc = document_rewriter.invoke(
            {
                "question": "如何使用RAG开发AI Agent？",
            }
        )

print(doc)